In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import shutil, csv, os, numpy as np
import pandas as pd
from glob import glob
from __future__ import print_function, division

import keras
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dense, Dropout, Flatten, Activation, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.optimizers import Adam, RMSprop, SGD, Adagrad
from keras.preprocessing import image
from keras.models import model_from_json

Using Theano backend.


In [2]:
path = 'data/'

In [3]:
batch_size = 64

# Todo

    1. Create train, valid and sample folders
    2. Move images into train, valid and sample sub folders
    3. Build neural network
    4. Submit

# Create train, valid and sample folders

In [7]:
data = []
reader = csv.reader(file('data/train.csv'))
reader.next()

for row in reader:
    data.append(row)

In [20]:
type(data), data[:5], data[0], len(data)

(list,
 [['0.png', '4'],
  ['1.png', '9'],
  ['2.png', '1'],
  ['3.png', '7'],
  ['4.png', '3']],
 ['0.png', '4'],
 49000)

In [14]:
os.mkdir(path + 'valid')
os.mkdir(path + 'sample')
os.mkdir(path + 'sample/train')
os.mkdir(path + 'sample/valid')

In [22]:
# Create sub folder for categories in train folder
for i in range(len(data)):
    if not os.path.isdir(path + 'train/' + data[i][1]):
        os.makedirs(path + 'train/' + data[i][1])

In [24]:
# Create sub folder for categories in valid folder
for i in range(len(data)):
    if not os.path.isdir(path + 'valid/' + data[i][1]):
        os.makedirs(path + 'valid/' + data[i][1])

In [25]:
# Create sub folder for categories in sample/train folder
for i in range(len(data)):
    if not os.path.isdir(path + 'sample/train/' + data[i][1]):
        os.makedirs(path + 'sample/train/' + data[i][1])
        
        
# Create sub folder for categories in valid folder
for i in range(len(data)):
    if not os.path.isdir(path + 'sample/valid/' + data[i][1]):
        os.makedirs(path + 'sample/valid/' + data[i][1])

# Move images into train, valid and sample sub folders

In [28]:
# Move images from train folder into subfolder
for i in range(len(data)):
    shutil.move(path + 'train/' + data[i][0], path + 'train/' + data[i][1])

In [52]:
# Move images from train/* folder into valid/*
dir_names = [i for i in os.listdir(path + 'train/') if os.path.isdir(os.path.join(path + 'train', i))]

for d in dir_names:
    g = glob(path + 'train/' + d + '/' + '*.png')
    shuf = np.random.permutation(g)
    
    for i in range(int(len(g) / 5)): shutil.move(shuf[i], path + 'valid/' + d +'/')

In [53]:
# Move images from train/* folder into sample/train/*
dir_names = [i for i in os.listdir(path + 'train/') if os.path.isdir(os.path.join(path + 'train', i))]

for d in dir_names:
    g = glob(path + 'train/' + d + '/' + '*.png')
    shuf = np.random.permutation(g)
    
    for i in range(int(len(g) / 10)): shutil.copy2(shuf[i], path + 'sample/train/' + d +'/')
        
        
# Move images from valid/* folder into sample/valid/*
dir_names = [i for i in os.listdir(path + 'valid/') if os.path.isdir(os.path.join(path + 'valid', i))]

for d in dir_names:
    g = glob(path + 'valid/' + d + '/' + '*.png')
    shuf = np.random.permutation(g)
    
    for i in range(int(len(g) / 10)): shutil.copy2(shuf[i], path + 'sample/valid/' + d +'/')

# Build Neural Network

In [4]:
def get_batches(path, gen = image.ImageDataGenerator(rescale = 1./255), shuffle = True, batch_size = batch_size, class_mode = 'categorical'):
    return gen.flow_from_directory(path, target_size = (28, 28), class_mode = class_mode, shuffle = shuffle, batch_size = batch_size)

In [5]:
batches = get_batches(path + 'sample/train/', batch_size = batch_size)
val_batches = get_batches(path + 'sample/valid/', batch_size = batch_size)

Found 3917 images belonging to 10 classes.
Found 975 images belonging to 10 classes.


In [10]:
model = Sequential()

model.add(Convolution2D(32, 3, 3, input_shape=(3, 28, 28), border_mode='same', activation='relu'))
model.add(Convolution2D(32, 3, 3, input_shape=(3, 28, 28), border_mode='same', activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [59]:
keras.optimizers.lr = 0.0001

In [60]:
model.fit_generator(batches, samples_per_epoch = batches.nb_sample, nb_epoch = 1,
                validation_data = val_batches, nb_val_samples = val_batches.nb_sample)

Epoch 1/1
3917/3917 [==============================] - 42s - loss: 0.8137 - acc: 0.7299 - val_loss: 0.2049 - val_acc: 0.9385

In [61]:
model.fit_generator(batches, samples_per_epoch = batches.nb_sample, nb_epoch = 2,
                validation_data = val_batches, nb_val_samples = val_batches.nb_sample)

Epoch 1/2
3917/3917 [==============================] - 38s - loss: 0.1802 - acc: 0.9451 - val_loss: 0.1142 - val_acc: 0.9733

In [62]:
# serialize model to JSON
model_json = model.to_json()
with open(path + "results/ft_3.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights(path + "results/ft_3.h5")
print("Saved model to disk")

Saved model to disk


In [63]:
# load json and create model
json_file = open(path + 'results/ft_3.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights(path + "results/ft_3.h5")
print("Loaded model from disk")

Loaded model from disk


In [64]:
keras.optimizers.lr = 0.01

In [65]:
model.fit_generator(batches, samples_per_epoch = batches.nb_sample, nb_epoch = 1,
                validation_data = val_batches, nb_val_samples = val_batches.nb_sample)

Epoch 1/1
3917/3917 [==============================] - 40s - loss: 0.0723 - acc: 0.9778 - val_loss: 0.1111 - val_acc: 0.9651

In [66]:
keras.optimizers.lr = 0.001

In [68]:
model.fit_generator(batches, samples_per_epoch = batches.nb_sample, nb_epoch = 2,
                validation_data = val_batches, nb_val_samples = val_batches.nb_sample)

Epoch 1/2
3917/3917 [==============================] - 40s - loss: 0.0366 - acc: 0.9900 - val_loss: 0.1169 - val_acc: 0.9662

In [69]:
# serialize model to JSON
model_json = model.to_json()
with open(path + "results/ft_8.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights(path + "results/ft_8.h5")
print("Saved model to disk")

Saved model to disk


In [70]:
# load json and create model
json_file = open(path + 'results/ft_8.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights(path + "results/ft_8.h5")
print("Loaded model from disk")

Loaded model from disk


In [71]:
keras.optimizers.lr = 0.0001

In [72]:
model.fit_generator(batches, samples_per_epoch = batches.nb_sample, nb_epoch = 1,
                validation_data = val_batches, nb_val_samples = val_batches.nb_sample)

Epoch 1/1
3917/3917 [==============================] - 40s - loss: 0.0407 - acc: 0.9883 - val_loss: 0.0592 - val_acc: 0.9836

In [73]:
model.fit_generator(batches, samples_per_epoch = batches.nb_sample, nb_epoch = 2,
                validation_data = val_batches, nb_val_samples = val_batches.nb_sample)

Epoch 1/2
3917/3917 [==============================] - 41s - loss: 0.0215 - acc: 0.9939 - val_loss: 0.0999 - val_acc: 0.9744